In [1]:
import prop
import pandas as pd
import matplotlib.pyplot as plt
from modules import pump, regen, heater, turbine, condenser

#Таблица с потоками и блоками:
streams = pd.read_excel("streams.xlsx",index_col=0)
blocks  = pd.read_excel("blocks.xlsx" ,index_col=0)

#Параметры нагревающей среды:
gas  = 'REFPROP::N2[0.78]&O2[0.15]&CO2[0.02]&H2O[0.05]' #Состав газа
Tin  = 183.6                                           #Температура газов на входе
Pgas = 0.1                                             #Давление газов
Ggas = 509                                             #Расход газов
Tout = 80                                              #Температура газов на выходе

#Параметры охлаждающей среды:
fluidcond  = "REFPROP::WATER" #Охлаждающая среда
Tfluidcond = 15               #Температура охл. среды
Pfluidcond = 0.5              #Давление охл. среды

#Параметры ОЦР:
fluid   = "REFPROP::R236ea"       #Рабочая среда цикла
KPDpump = 0.85                 #КПД насоса
KPDturb = 0.85                 #КПД турбины
DTheat  = 10                   #Минимальный напор в нагревателе
DTcond  = 5                    #Минимальный напор в конденсаторе
Pcond   = prop.t_q(30,0,fluid)['P']               #Давление в конденсаторе

#Первое приближение

dTreg   = 20
Ppump   = 3.5                 #Давление на выходе из насоса




dPreg1  = 0.01                #Потери по горячему в МПа
for i in range(1):
    dPreg2  = dPreg1                #Потери по холодному
    streams = pd.read_excel("streams.xlsx",index_col=0)
    blocks  = pd.read_excel("blocks.xlsx" ,index_col=0)

    #Внесение входных данных в таблицу:
    streams.loc["IN-HEAT", "T":"Q"] = [Tin,Pgas,prop.t_p(Tin,Pgas,gas)["H"],prop.t_p(Tin,Pgas,gas)["S"],Ggas,1]
    streams.loc["IN-COND", "T":"Q"] = [Tfluidcond,Pfluidcond,prop.t_p(Tfluidcond,Pfluidcond,fluidcond)["H"],prop.t_p(Tfluidcond,Pfluidcond,fluidcond)["S"],1000,0]

    #Начало расчёта - задание расхода на первой итерации:
    G = Ggas*(prop.t_p(Tin,Pgas,gas)["H"]-prop.t_p(Tout,Pgas,gas)["H"])/(prop.t_p(Tin,Ppump,fluid)["H"]-prop.t_p(Tout,Ppump,fluid)["H"])

    #Поток перед насосом:
    streams.loc["COND-PUMP", "T":"Q"] = [prop.p_q(Pcond,0,fluid)["T"],Pcond,prop.p_q(Pcond,0,fluid)['H'],prop.p_q(Pcond,0,fluid)['S'],G,0]

    #Итеративный расчет для сведения баланса:
    for i in range(1000):
        #Расчет цикла:
        pump.calc     ("COND-PUMP", "PUMP-REG",                          Ppump,  KPDpump,     fluid, streams, blocks)
        regen.calc    ("TURB-REG",  "REG-COND",  "PUMP-REG","REG-HEAT",  dTreg, dPreg1,dPreg2,fluid, streams, blocks)
        heater.calc   ("IN-HEAT",   "HEAT-OUT",  "REG-HEAT","HEAT-TURB", Tout,   DTheat, gas, fluid, streams, blocks)
        turbine.calc  ("HEAT-TURB", "TURB-REG",                          Pcond+dPreg1,  KPDturb,fluid, streams, blocks)
        regen.calc    ("TURB-REG",  "REG-COND",  "PUMP-REG","REG-HEAT",  dTreg, dPreg1,dPreg2,fluid, streams, blocks)
        condenser.calc("REG-COND",  "COND-PUMP", "IN-COND" ,"COND-OUT",  DTcond, fluidcond,   fluid, streams, blocks)
        Qbalance = blocks.loc['HEATER','Q']+blocks.loc['PUMP','N']-blocks.loc['CONDENSER','Q']-blocks.loc['TURBINE','N']
#         print(Qbalance)
        if abs(Qbalance) < 1:
            break

    money = regen.constr("TURB-REG",  "REG-COND",  "PUMP-REG","REG-HEAT",  dPreg1,dPreg1,streams, blocks)['money']
        
    print(dTreg, dPreg1, money)





dPreg1  = 0.02                #Потери по горячему в МПа
for i in range(5):
    dPreg2  = dPreg1                #Потери по холодному

    streams = pd.read_excel("streams.xlsx",index_col=0)
    blocks  = pd.read_excel("blocks.xlsx" ,index_col=0)

    #Внесение входных данных в таблицу:
    streams.loc["IN-HEAT", "T":"Q"] = [Tin,Pgas,prop.t_p(Tin,Pgas,gas)["H"],prop.t_p(Tin,Pgas,gas)["S"],Ggas,1]
    streams.loc["IN-COND", "T":"Q"] = [Tfluidcond,Pfluidcond,prop.t_p(Tfluidcond,Pfluidcond,fluidcond)["H"],prop.t_p(Tfluidcond,Pfluidcond,fluidcond)["S"],1000,0]

    #Начало расчёта - задание расхода на первой итерации:
    G = Ggas*(prop.t_p(Tin,Pgas,gas)["H"]-prop.t_p(Tout,Pgas,gas)["H"])/(prop.t_p(Tin,Ppump,fluid)["H"]-prop.t_p(Tout,Ppump,fluid)["H"])

    #Поток перед насосом:
    streams.loc["COND-PUMP", "T":"Q"] = [prop.p_q(Pcond,0,fluid)["T"],Pcond,prop.p_q(Pcond,0,fluid)['H'],prop.p_q(Pcond,0,fluid)['S'],G,0]

    #Итеративный расчет для сведения баланса:
    for i in range(1000):
        #Расчет цикла:
        pump.calc     ("COND-PUMP", "PUMP-REG",                          Ppump,  KPDpump,     fluid, streams, blocks)
        regen.calc    ("TURB-REG",  "REG-COND",  "PUMP-REG","REG-HEAT",  dTreg, dPreg1,dPreg2,fluid, streams, blocks)
        heater.calc   ("IN-HEAT",   "HEAT-OUT",  "REG-HEAT","HEAT-TURB", Tout,   DTheat, gas, fluid, streams, blocks)
        turbine.calc  ("HEAT-TURB", "TURB-REG",                          Pcond+dPreg1,  KPDturb,fluid, streams, blocks)
        regen.calc    ("TURB-REG",  "REG-COND",  "PUMP-REG","REG-HEAT",  dTreg, dPreg1,dPreg2,fluid, streams, blocks)
        condenser.calc("REG-COND",  "COND-PUMP", "IN-COND" ,"COND-OUT",  DTcond, fluidcond,   fluid, streams, blocks)
        Qbalance = blocks.loc['HEATER','Q']+blocks.loc['PUMP','N']-blocks.loc['CONDENSER','Q']-blocks.loc['TURBINE','N']
#         print(Qbalance)
        if abs(Qbalance) < 1:
            break

    money = regen.constr("TURB-REG",  "REG-COND",  "PUMP-REG","REG-HEAT",  dPreg1,dPreg1,streams, blocks)['money']
        
    print(dTreg, dPreg1, money)
    dPreg1 = dPreg1+0.02


20 0.01 6304210
20 0.02 5654947
20 0.04 5451260
20 0.06 5024972
20 0.08 5125267
20 0.1 5100836
